In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process


In [2]:
def read_file(url):
    '''this function is to read file from local path.'''
    with open(url) as f:
        df = pd.read_csv(url, na_values='Nan')
        df.set_index('userid',inplace=True)
        
    return df

In [3]:
#user_rating_matrix = read_file('../data/user_rating_metrix.csv')

In [4]:
def update_new_user_to_user_rating_matrix(user_rating, user_rating_matrix):
    """
    convert dict of user_ratings to a user_data frame with the same column names as the user_item_matrix
    that can be easily appended to the user_item_matrix
    """            
    
    user_vector = pd.Series(np.nan, index=user_rating_matrix.columns)
    user_seened_movie = []
    for movie in list(user_rating.keys()): 
        matched_title = process.extractOne(movie, user_vector.index)[0]
        user_vector[matched_title] = user_rating[movie]
        user_seened_movie.append(process.extractOne(movie, user_vector.index)[0])
    
    new_user_df = user_vector.to_frame().rename(columns={0:np.max(user_rating_matrix.index)+1}).T
    
    return user_rating_matrix.append(new_user_df), user_seened_movie, new_user_df

In [5]:
#user_rating_matrix_update, user_seened_movie,new_user_df = update_new_user_to_user_rating_matrix(user_rating, user_rating_matrix)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
def nor(scaler, df):
    df_scale = pd.DataFrame(scaler.fit_transform(df) , columns= df.columns, index= df.index)
    return df_scale   

In [8]:
#user_rating_matrix_update_scaled=nor(scaler=StandardScaler(), df=user_rating_matrix_update.fillna(0))

In [9]:
def get_similarity(cosine, df):
    similarity_movie=pd.DataFrame(cosine_similarity(df.T),columns= df.columns, index=df.columns)
    similarity_user =pd.DataFrame (cosine_similarity(df),columns=df.index,  index= df.index)
    return similarity_movie,similarity_user.sort_values(similarity_user.columns[-1],axis=0, ascending=False)

In [10]:
#similarity_movie, similarity_user=get_similarity(cosine=cosine_similarity, df=user_rating_matrix_update_scaled)

In [11]:
def get_rec_movies(user_rating_matrix_update,similarity_user):
    
    return user_rating_matrix_update.loc[[i for i in similarity_user[0:10].index.values]]    

In [12]:
#rec_movies = get_rec_movies(user_rating_matrix_update,similarity_user)

In [13]:
def recommender_sim():
    
    recommendations = []
    new_user_t=new_user_df.T
    new_user_t.loc[new_user_t[611]>=4].index.to_list()
#test = similarity_movie.sort_values("Empire of Dreams: The Story of the 'Star Wars' Trilogy",axis=0, ascending=False)
    for movie in new_user_t.loc[new_user_t[611]>=4].index.to_list():
        recommendations.append(similarity_movie.loc[(similarity_movie[movie]>=0.7)&(similarity_movie[movie]<1)].index.to_list())
    return recommendations
    

In [14]:
if __name__ == '__main__':
    
    user_rating_matrix = read_file('../data/user_rating_metrix.csv')
    user_rating = {
    'about time': 5,
    'queen': 5,
    'star wars': 5
    }
    
    user_rating_matrix_update, user_seened_movie,new_user_df = update_new_user_to_user_rating_matrix(user_rating, user_rating_matrix)
    user_rating_matrix_update_scaled=nor(scaler=StandardScaler(), df=user_rating_matrix_update.fillna(0))
    
    similarity_movie, similarity_user=get_similarity(cosine=cosine_similarity, df=user_rating_matrix_update_scaled)
    recommendation_cosin = recommender_sim()
    
    print(recommendation_cosin)

[[], ['Aelita: The Queen of Mars (Aelita)'], []]
